In [ ]:
!pip install geopy

In [45]:
!pip install xlrd

In [9]:
import geopy
import pandas as pd
import numpy as np
import requests
from time import sleep

In [2]:
geolocator = geopy.Nominatim()
location = geolocator.reverse("67.49595671, 64.02282715")

location.address

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


'9, 1-я Поселковая улица, Воркута, городской округ Воркута, Республика Коми, Северо-Западный федеральный округ, 169902, Россия'

In [4]:
data = pd.read_excel("/Users/eugenia/beeps_07_04.xlsx")

In [5]:
data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,idstd,lat_mask,lon_mask,survey,WBCode,type,id,a1,a2,h1,h2,h5,BMh2,BMh3,h9,City
0,0,0,1,656051,50.618305,36.577396,Russia2019,RUS,1,22,58,1,2,NaN,2,2,2,NaN,Белгород
1,1,1,2,656052,50.560638,36.672180,Russia2019,RUS,1,29,58,1,2,NaN,2,2,2,NaN,Белгород
2,2,2,3,656053,50.546829,36.645367,Russia2019,RUS,1,34,58,1,2,NaN,2,2,2,NaN,Белгород
3,3,3,4,656054,50.590969,36.608212,Russia2019,RUS,1,50,58,1,2,NaN,2,2,2,NaN,Белгород
4,4,4,5,656055,50.621201,36.568302,Russia2019,RUS,1,56,58,1,2,NaN,2,2,2,NaN,Белгород
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,1318,1318,1319,657369,54.745327,55.943592,Russia2019,RUS,1,699748,58,7,2,NaN,1,2,2,NaN,Уфа
1319,1319,1319,1320,657370,55.624409,37.601780,Russia2019,RUS,1,699752,58,1,2,NaN,2,2,2,NaN,Москва
1320,1320,1320,1321,657371,43.139450,131.902069,Russia2019,RUS,1,699770,58,4,2,NaN,2,2,2,NaN,Владивосток
1321,1321,1321,1322,657372,56.122303,94.570107,Russia2019,RUS,1,699775,58,5,2,NaN,2,1,2,NaN,Зеленогорск


In [7]:
coords = zip(data['lat_mask'], data["lon_mask"]) # packing pairs of coordinates into tuples
coords = list(coords) # each tuple is an element of a list => we can loop over these pairs
len(coords) # the same as the lenghts of the beeps dat

1323

In [14]:
idstd = []
h1 = []
h5 = []
BMh2 = []
name = []
lat = []
lon = []

for k in coords:
    url = f"http://api.geonames.org/findNearbyPlaceNameJSON?lat={coords[coords.index(k)][0]}&lng={coords[coords.index(k)][1]}&radius=40&lang=ru&username=eatarnikova"
    page = requests.get(url)
    json = page.json()
    for i in json['geonames']: 
        name.append(json['geonames'][json['geonames'].index(i)]['name'])
        lat.append(json['geonames'][json['geonames'].index(i)]['lat'])
        lon.append(json['geonames'][json['geonames'].index(i)]['lng'])
        idstd.append(list(data['idstd'])[coords.index(k)])
        h1.append(list(data['h1'])[coords.index(k)])
        h5.append(list(data['h5'])[coords.index(k)])
        BMh2.append(list(data['BMh2'])[coords.index(k)])
    if coords.index(k) == 250 or coords.index(k) == 500 or coords.index(k) == 750 or coords.index(k) == 1000 or coords.index(k) == 1250:
        sleep(3600)

In [23]:
data = pd.DataFrame({'name':name, 'lat':lat, 'lng':lon, 'idstd':idstd, 'h1':h1, 'h5': h5, 'BMh2': BMh2})

In [25]:
writer = pd.ExcelWriter('beeps_22_04.xlsx', engine='xlsxwriter')

data.to_excel(writer, sheet_name='Sheet1')

writer.save()

In [21]:
len(lat)

13230